In [1]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance
from qiskit import IBMQ
import math

In [2]:
def register_cnot(circuit,x,t,type,n_ctrl):
    if type=='0':
        for i in range(0,n_ctrl):
            circuit.x(x[i])
        circuit.mcx(x,t)
        for i in range(0,n_ctrl):
            circuit.x(x[i])
    else:
        circuit.mcx(x,t)

In [3]:
def doubly_controlled_halving(circuit,ca,cb,x,n):
    sub_x = QuantumRegister(n)
    sub_circuit = QuantumCircuit(sub_x,name='/2')
    ECC_library.binary_halving_cyclic(sub_circuit,sub_x,n)
    sub_inst=sub_circuit.to_gate().control().control()
    circuit.append(sub_inst,ca[0:1]+cb[0:1]+x[0:n])
    
def doubly_controlled_doubling(circuit,ca,cb,x,n):
    sub_x = QuantumRegister(n)
    sub_circuit = QuantumCircuit(sub_x,name='*2')
    ECC_library.binary_doubling_cyclic(sub_circuit,sub_x,n)
    sub_inst=sub_circuit.to_gate().control().control()
    circuit.append(sub_inst,ca[0:1]+cb[0:1]+x[0:n])
    
def doubly_controlled_addition(circuit,ca,cb,x,y,n):
    sub_x = QuantumRegister(n)
    sub_y = QuantumRegister(n)
    sub_circuit = QuantumCircuit(sub_x,sub_y,name='+')
    ECC_library.addition_without_carry(sub_circuit,sub_x,sub_y,n)
    sub_inst=sub_circuit.to_gate().control().control()
    circuit.append(sub_inst,ca[0:1]+cb[0:1]+x[0:n]+y[0:n])
    
def doubly_controlled_subtraction(circuit,ca,cb,x,y,n):
    sub_x = QuantumRegister(n)
    sub_y = QuantumRegister(n)
    sub_circuit = QuantumCircuit(sub_x,sub_y,name='-')
    ECC_library.addition_without_carry(sub_circuit,sub_x,sub_y,n)
    sub_inst=sub_circuit.to_gate().inverse().control().control()
    circuit.append(sub_inst,ca[0:1]+cb[0:1]+x[0:n]+y[0:n])

In [4]:
def block(circuit,u,v,ca,s,r,cb,cc,m,n):
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    circuit.x(u[0])
    circuit.cx(u[0],cc)
    circuit.x(u[0])
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    circuit.x(v[0])
    circuit.x(cc)
    circuit.ccx(v[0],cc,m)
    circuit.x(cc)
    circuit.x(v[0])
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    circuit.cx(cc,cb)
    circuit.cx(m,cb)
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    ECC_library.subtraction(circuit,u,v,ca,n)
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    circuit.x(cb)
    circuit.ccx(ca,cb,cc)
    circuit.ccx(ca,cb,m)
    circuit.x(cb)
    circuit.cx(m,cb)
    circuit.cx(cc,cb)
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    ECC_library.addition(circuit,u,v,ca,n)
    circuit.barrier(u,v,ca,s,r,cb,cc,m)
    circuit.x(m)
    doubly_controlled_halving(circuit,cc,m,u,n)
    doubly_controlled_doubling(circuit,cc,m,s,n)
    circuit.x(m)
    circuit.x(cc)
    doubly_controlled_halving(circuit,cc,m,v,n)
    doubly_controlled_doubling(circuit,cc,m,r,n)
    circuit.x(cc)
    doubly_controlled_subtraction(circuit,cc,m,v,u,n)
    doubly_controlled_addition(circuit,cc,m,s,r,n)
    doubly_controlled_halving(circuit,cc,m,u,n)
    doubly_controlled_doubling(circuit,cc,m,s,n)
    circuit.x(m)
    circuit.x(cc)
    doubly_controlled_subtraction(circuit,cc,m,u,v,n)
    doubly_controlled_addition(circuit,cc,m,r,s,n)
    doubly_controlled_halving(circuit,cc,m,v,n)
    doubly_controlled_doubling(circuit,cc,m,r,n)
    circuit.x(m)
    circuit.x(cc)
    circuit.cx(r[0],cc)

In [5]:
a = 1
p = 1
s = 8
r = 3
n = 4
s_a=bin(int(a))[2:].zfill(n)
s_p=bin(int(p))[2:].zfill(n)
s_r=bin(int(r))[2:].zfill(n)
s_s=bin(int(s))[2:].zfill(n)
s_p=s_p[::-1]


u = QuantumRegister(n,'u')
v = QuantumRegister(n,'v')
ca = QuantumRegister(1,'ca')
s = QuantumRegister(n,'s')
r = QuantumRegister(n,'r')
cb = QuantumRegister(1,'cb')
cc = QuantumRegister(1,'cc')
m = QuantumRegister(1,'m')
clas_u = ClassicalRegister(n)
clas_v = ClassicalRegister(n)
clas_s = ClassicalRegister(n)
clas_r = ClassicalRegister(n)
circuit = QuantumCircuit(u,v,ca,s,r,cb,cc,m,clas_u,clas_v,clas_s,clas_r)
for i in range(0,n):
    if s_p[i]=='1':
        circuit.x(u[i])
    if s_a[i] == '1':
        circuit.x(v[n-1-i])
    if s_r[i] == '1':
        circuit.x(r[n-1-i])
    if s_s[i] == '1':
        circuit.x(s[n-1-i])
        
block(circuit,u,v,ca,s,r,cb,cc,m,n)

for i in range(0,n):
    circuit.measure(u[i],clas_u[i])
for i in range(0,n):
    circuit.measure(v[i],clas_v[i])
for i in range(0,n):
    circuit.measure(s[i],clas_s[i])
for i in range(0,n):
    circuit.measure(r[i],clas_r[i])

In [6]:
IBMQ.load_account();
provider = IBMQ.get_provider('ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')
simulation = execute(circuit, backend=backend ,shots=10)
sim_result = simulation.result()
out=list(sim_result.get_counts().keys())[0]
output=out.split(' ')
output_u = output[3]
output_v = output[2]
output_s = output[1]
output_r = output[0]
res_u = int(output_u, 2)
res_v = int(output_v, 2)
res_s = int(output_s, 2)
res_r = int(output_r, 2)
print('res_u:',res_u,'res_v:',res_v,'res_s:',res_s,'res_r:',res_r)

res_u: 1 res_v: 0 res_s: 11 res_r: 6
